In [1]:
# Load from parent directory if not installed
import importlib
import os

if not importlib.util.find_spec("sammo"):
    import sys

    sys.path.append("../")
os.environ["CACHE_FILE"] = "cache/quickstart.tsv"

# 🚀 Quick Start

At the core of SAMMO are symbolic prompt programs. This tutorial will show you a few simple programs.

To run this example, you need API credentials to an OpenAI API compatible model. 

```{note}
You an run all of these as live notebooks on Binder. Simply click the rocket icon above.
```
Below, we will use ChatGPT-3.5 and also cache any requests made.

In [2]:
# %load -r 3:25 _init.py
import pathlib
import sammo
from sammo.runners import OpenAIChat
from sammo.base import Template, EvaluationScore
from sammo.components import Output, GenerateText, ForEach, Union
from sammo.extractors import ExtractRegex
from sammo.data import DataTable
import json
import requests
import os

if not 'OPENAI_API_KEY' in os.environ:
    raise ValueError("Please set the environment variable 'OPENAI_API_KEY'.")

_ = sammo.setup_logger("WARNING")  # we're only interested in warnings for now

runner = OpenAIChat(
    model_id="gpt-3.5-turbo",
    api_config={"api_key": os.environ['OPENAI_API_KEY']},
    cache=os.getenv("CACHE_FILE", "cache.tsv"),
    timeout=30,
)

Let's write our first symbolic prompt program (SPP)! How about a quick 'Hello World?'?

In [3]:
spp_hello_world = Output(GenerateText("Hello World!"))
spp_hello_world.run(runner)

+---------+------------------------------------+
| input   | output                             |
+=========+====================================+
| None    | Hello! How can I assist you today? |
+---------+------------------------------------+
Constants: None

## Writing symbolic prompt programs
Okay, let's move on to a more interesting example. For a list of countries, we want the top reason to visit:

In [4]:
COUNTRIES = ["Switzerland", "Morocco", "Tanzania", "Indonesia", "Peru"]

reason_to_visit = GenerateText(
    Template("What is the top reason to visit {{input}} in one sentence?")
)
Output(reason_to_visit).run(runner, COUNTRIES)[:2]

minibatches[###################################################################################]5/5[00:00<??:??, 0.00it/s]


+-------------+--------------------------------------------------------------+
| input       | output                                                       |
+=============+==============================================================+
| Switzerland | The stunning natural beauty of the Swiss Alps and crystal-   |
|             | clear lakes make Switzerland a must-visit destination for    |
|             | outdoor enthusiasts and nature lovers.                       |
+-------------+--------------------------------------------------------------+
| Morocco     | The top reason to visit Morocco is to experience the vibrant |
|             | culture, stunning architecture, and diverse landscapes of    |
|             | this North African country.                                  |
+-------------+--------------------------------------------------------------+
Constants: None

What happens under the hood is that SAMMO parallizes the execution across all inputs automatically! 

Let's add the best time to visit to it and combine both pieces of information.

In [5]:
when_to_visit = GenerateText(
    Template(
        "Which season is the best time to visit {{input}}? Answer in one sentence."
    )
)
country_pages = Template(
    "# {{input}}\n{{reason}}\n\n## When to Visit\n{{when}}",
    reason=reason_to_visit,
    when=when_to_visit,
)
written_pages = Output(country_pages).run(runner, COUNTRIES)
written_pages[:2]

minibatches[#################################################################################]5/5[00:00<00:00, 312.50it/s]


+-------------+-------------------------------------------------------------+
| input       | output                                                      |
+=============+=============================================================+
| Switzerland | # Switzerland The stunning natural beauty of the Swiss Alps |
|             | and crystal-clear lakes make Switzerland a must-visit       |
|             | destination for outdoor enthusiasts and nature lovers.  ##  |
|             | When to Visit The best time to visit Switzerland is during  |
|             | the summer months (June to August) when the weather is warm |
|             | and ideal for outdoor activities.                           |
+-------------+-------------------------------------------------------------+
| Morocco     | # Morocco The top reason to visit Morocco is to experience  |
|             | the vibrant culture, stunning architecture, and diverse     |
|             | landscapes of this North African country.  ## Wh

Nice! To see what operations SAMMO executed under the hood, we can use `plot_call_trace()` to show all intermediate calls. You can click on each node to get more info.

In [9]:
written_pages.outputs[0].plot_call_trace()

## Recap
Let's talk about some of the key concepts from SAMMO we have used:

1. We constructed a **symbolic prompt program** — a dynamic prompt that is re-used for different inputs.
2. This SPP has a structure which was constructed by nesting **components** from SAMMO.
3. To get the **output**, we call `run()` on the Output component which returns a DataTable.
4. SAMMO **parallelized** execution for us on the input data — no extra work was needed! 